In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import scipy.special

In [5]:
orginal_data = pd.read_csv("oasis_longitudinal.csv")
mu_eTIV, sigma_eTIV = orginal_data['eTIV'].mean(), orginal_data['eTIV'].var()
mu_MMSE, sigma_MMSE = orginal_data['MMSE'].mean(), orginal_data['MMSE'].var()
mu_EDUC, sigma_EDUC = orginal_data['EDUC'].mean(), orginal_data['EDUC'].var()
age_range = (60, 100)

def build_lstm_rnn(INPUT_SHAPE):
    lstm_rnn = tf.keras.Sequential()
    lstm_rnn.add(tf.keras.layers.LSTM(100, return_sequences=True, input_shape=INPUT_SHAPE))
    lstm_rnn.add(tf.keras.layers.LSTM(50, return_sequences=False))
    lstm_rnn.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    lstm_rnn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return lstm_rnn

accuracy_list = []
recall_list = []
precision_list = []

In [6]:
N = 100
m = 5

Y = eta = np.zeros((N, m))
Age = np.random.choice(range(age_range[0], age_range[1] + 1), size=N)
MMSE = np.round(np.random.normal(mu_MMSE, sigma_MMSE, size=N))
EDUC = np.round(np.random.normal(mu_EDUC, sigma_EDUC, size=N))
Sex = np.random.binomial(1, 0.2, size=N)
eTIV = np.round(np.random.normal(mu_eTIV, sigma_eTIV, size=N))
ASF = np.random.normal(size=N)
nWBV = np.random.normal(size=N)
Visit = np.round(np.linspace(0, 4, num=m))

B = np.array([0.1, -0.1, 0.2, -0.2, 0.3, 0.3, 0.1, 0.1, 0.2, 0.1])
b = np.random.normal(0, 0.5, size = N)

for i in range(N):
    for j in range(m):
        eta[i, j] = B[0] + B[1] * Visit[j] + B[2] * Age[i] + B[3] * MMSE[i] + B[4] * EDUC[i] + B[5] * Sex[i] + \
                    B[6] * eTIV[i] + B[7] * ASF[i] + B[8] * nWBV[i] + b[i]
        p = scipy.special.expit(eta[i, j])
        Y[i, j] = np.random.binomial(1, p)



mat = np.empty((N * m, 10))
p = 0
for i in range(N):
    jk = 0
    for jj in range(p, p + m):
        mat[jj, 0] = i + 1
        mat[jj, 1] = Visit[jk]
        mat[jj, 2] = Age[i]
        mat[jj, 3] = MMSE[i]
        mat[jj, 4] = EDUC[i]
        mat[jj, 5] = Sex[i]
        mat[jj, 6] = eTIV[jk]
        mat[jj, 7] = ASF[jk]
        mat[jj, 8] = nWBV[jk]
        mat[jj, 9] = Y[i, jk]
        jk += 1
    p += m
# Convert the `mat` array to a DataFrame
df_mat = pd.DataFrame(mat, columns=['ID', 'Visit', 'Age', 'MMSE', 'EDUC', 'Sex', 'eTIV', 'ASF', 'nWBV', 'Y'])
df = pd.get_dummies(df_mat, columns=["Visit"], prefix="Visit")

In [12]:
df_mat

,ID,Visit,Age,MMSE,EDUC,Sex,eTIV,ASF,nWBV,Y
0,1.0,0.0,92.0,36.0,16.0,0.0,31089.0,-1.068323,0.285253,1.0
1,1.0,1.0,92.0,36.0,16.0,0.0,-9972.0,-0.010969,0.657046,1.0
2,1.0,2.0,92.0,36.0,16.0,0.0,60888.0,0.805799,1.079854,1.0
3,1.0,3.0,92.0,36.0,16.0,0.0,19312.0,-0.042708,-0.209555,1.0
4,1.0,4.0,92.0,36.0,16.0,0.0,41089.0,-0.762549,-1.646706,1.0
...,...,...,...,...,...,...,...,...,...,...
495,100.0,0.0,100.0,13.0,10.0,1.0,31089.0,-1.068323,0.285253,1.0
496,100.0,1.0,100.0,13.0,10.0,1.0,-9972.0,-0.010969,0.657046,1.0
497,100.0,2.0,100.0,13.0,10.0,1.0,60888.0,0.805799,1.079854,1.0
498,100.0,3.0,100.0,13.0,10.0,1.0,19312.0,-0.042708,-0.209555,1.0


In [27]:
df_mat[(df_mat["Visit"] == 4) & (df_mat["Y"] == 0)]

,ID,Visit,Age,MMSE,EDUC,Sex,eTIV,ASF,nWBV,Y
9,2.0,4.0,66.0,32.0,10.0,1.0,41089.0,-0.762549,-1.646706,0.0
29,6.0,4.0,72.0,48.0,16.0,0.0,41089.0,-0.762549,-1.646706,0.0
34,7.0,4.0,82.0,16.0,12.0,0.0,41089.0,-0.762549,-1.646706,0.0
49,10.0,4.0,92.0,12.0,20.0,0.0,41089.0,-0.762549,-1.646706,0.0
64,13.0,4.0,70.0,37.0,24.0,0.0,41089.0,-0.762549,-1.646706,0.0
79,16.0,4.0,91.0,17.0,10.0,0.0,41089.0,-0.762549,-1.646706,0.0
94,19.0,4.0,66.0,27.0,10.0,0.0,41089.0,-0.762549,-1.646706,0.0
99,20.0,4.0,88.0,27.0,15.0,1.0,41089.0,-0.762549,-1.646706,0.0
109,22.0,4.0,70.0,36.0,1.0,0.0,41089.0,-0.762549,-1.646706,0.0
114,23.0,4.0,97.0,36.0,6.0,0.0,41089.0,-0.762549,-1.646706,0.0


In [ ]:
RANDOM_STATE = 13
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state=RANDOM_STATE)
split = splitter.split(df, groups=df['ID'])
train_indexes, test_indexes = next(split)

X_train = df.iloc[train_indexes]
X_test = df.iloc[test_indexes]
age_scaler = MinMaxScaler()
educ_scaler = MinMaxScaler()
mmse_scaler = MinMaxScaler()
etiv_scaler = MinMaxScaler()

age_scaler.fit(X_train["Age"].to_numpy().reshape(-1, 1))
educ_scaler.fit(X_train["EDUC"].to_numpy().reshape(-1, 1))
mmse_scaler.fit(X_train["MMSE"].to_numpy().reshape(-1, 1))
etiv_scaler.fit(X_train["eTIV"].to_numpy().reshape(-1, 1))

X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
X_train["MMSE"] = mmse_scaler.transform(X_train["MMSE"].to_numpy().reshape(-1, 1))
X_train["eTIV"] = etiv_scaler.transform(X_train["eTIV"].to_numpy().reshape(-1, 1))

X_test["Age"] = age_scaler.transform(X_test["Age"].to_numpy().reshape(-1, 1))
X_test["EDUC"] = educ_scaler.transform(X_test["EDUC"].to_numpy().reshape(-1, 1))
X_test["MMSE"] = mmse_scaler.transform(X_test["MMSE"].to_numpy().reshape(-1, 1))
X_test["eTIV"] = mmse_scaler.transform(X_test["eTIV"].to_numpy().reshape(-1, 1))

In [25]:
# issue
X_train.ID.unique().shape

(80,)

In [83]:
visit_4_train = X_train[X_train["Visit_4.0"] == 1][["ID", "Y"]]
y_train_final = []
for id in X_train["ID"].unique():
    y_train_final.append(visit_4_train[visit_4_train["ID"] == id]["Y"].values[0])

visit_4_test = X_test[X_test["Visit_4.0"] == 1][["ID", "Y"]]
y_test_final = []
for id in X_test["ID"].unique():
    y_test_final.append(visit_4_test[visit_4_test["ID"] == id]["Y"].values[0])


In [ ]:

y_test_final = []
for id in X_test["ID"]:
    y_test_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

y_train_super_final = pd.DataFrame({
"Group": y_train_final})
y_test_super_final = pd.DataFrame({
"Group": y_test_final})
X_test_final = X_test.drop(["ID"], axis = 1)
X_train_final = X_train.drop(["ID"], axis = 1)
rows_n = 5

X_train_super_final = np.reshape(X_train_final.to_numpy(),(X_train_final.shape[0]//5,X_train_final.shape[1],rows_n))
X_test_super_final = np.reshape(X_test_final.to_numpy(),(X_test_final.shape[0]//5,X_train_final.shape[1],rows_n))
lstm_rnn = build_lstm_rnn((X_train_super_final.shape[1],X_train_super_final.shape[2]) )
lstm_rnn.fit(X_train_super_final, y_train_super_final[0:int(len(X_train_super_final))], epochs = 100)
not_final_lstm_rnn_prediction = lstm_rnn.predict(X_test_super_final)
lstm_rnn_prediction = np.where(not_final_lstm_rnn_prediction > 0.5, 1, 0)
lstm_rnn_conf = confusion_matrix(y_test_super_final[0:int(len(lstm_rnn_prediction))],lstm_rnn_prediction)
lstm_rnn_plot_conf = ConfusionMatrixDisplay(lstm_rnn_conf)
lstm_rnn_plot_conf.plot()
TP=lstm_rnn_conf[1,1]
FP=lstm_rnn_conf[0,1]
TN=lstm_rnn_conf[0,0]
FN=lstm_rnn_conf[1,0]
accuracy=(TP+TN)/(TP+FP+FN+TN)
recall =TP/(TP+FN)
precision =TP/(TP+TN)
accuracy_list.append(accuracy)
recall_list.append(recall)
precision_list.append(precision)

print("Accuracy List:", accuracy_list)
print("Recall List:", recall_list)
print("Precision List:", precision_list)

In [ ]:
for _ in range(10):
    N = 100
    m = 5

    Y = eta = np.zeros((N, m))
    Age = np.random.choice(range(age_range[0], age_range[1] + 1), size=N)
    MMSE = np.round(np.random.normal(mu_MMSE, sigma_MMSE, size=N))
    EDUC = np.round(np.random.normal(mu_EDUC, sigma_EDUC, size=N))
    Sex = np.random.binomial(1, 0.2, size=N)
    eTIV = np.round(np.random.normal(mu_eTIV, sigma_eTIV, size=N))
    ASF = np.random.normal(size=N)
    nWBV = np.random.normal(size=N)
    Visit = np.round(np.linspace(0, 4, num=m))

    B = np.array([0.1, -0.1, 0.2, -0.2, 0.3, 0.3, 0.1, 0.1, 0.2, 0.1])
    b = np.random.normal(0, 0.5, size=N)

    for i in range(N):
        for j in range(m):
            eta[i, j] = B[0] + B[1] * Visit[j] + B[2] * Age[i] + B[3] * MMSE[i] + B[4] * EDUC[i] + B[5] * Sex[i] + \
                        B[6] * eTIV[i] + B[7] * ASF[i] + B[8] * nWBV[i] + b[i]
            p = scipy.special.expit(eta[i, j])
            Y[i, j] = np.random.binomial(1, p)

   

    mat = np.empty((N * m, 10))
    p = 0
    for i in range(N):
        jk = 0
        for jj in range(p, p + m):
            mat[jj, 0] = i + 1
            mat[jj, 1] = Visit[jk]
            mat[jj, 2] = Age[i]
            mat[jj, 3] = MMSE[i]
            mat[jj, 4] = EDUC[i]
            mat[jj, 5] = Sex[i]
            mat[jj, 6] = eTIV[jk]
            mat[jj, 7] = ASF[jk]
            mat[jj, 8] = nWBV[jk]
            mat[jj, 9] = Y[i, jk]
            jk += 1
        p += m

    # Convert the `mat` array to a DataFrame
    df_mat = pd.DataFrame(mat, columns=['ID', 'Visit', 'Age', 'MMSE', 'EDUC', 'Sex', 'eTIV', 'ASF', 'nWBV', 'Y'])
    df = pd.get_dummies(df_mat, columns=["Visit"], prefix="Visit")
    RANDOM_STATE = 13
    splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state=RANDOM_STATE)
    split = splitter.split(df, groups=df['ID'])
    train_indexes, test_indexes = next(split)

    X_train = df.iloc[train_indexes]
    X_test = df.iloc[test_indexes]
    age_scaler = MinMaxScaler()
    educ_scaler = MinMaxScaler()
    mmse_scaler = MinMaxScaler()
    etiv_scaler = MinMaxScaler()

    age_scaler.fit(X_train["Age"].to_numpy().reshape(-1, 1))
    educ_scaler.fit(X_train["EDUC"].to_numpy().reshape(-1, 1))
    mmse_scaler.fit(X_train["MMSE"].to_numpy().reshape(-1, 1))
    etiv_scaler.fit(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
    X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
    X_train["MMSE"] = mmse_scaler.transform(X_train["MMSE"].to_numpy().reshape(-1, 1))
    X_train["eTIV"] = etiv_scaler.transform(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_test["Age"] = age_scaler.transform(X_test["Age"].to_numpy().reshape(-1, 1))
    X_test["EDUC"] = educ_scaler.transform(X_test["EDUC"].to_numpy().reshape(-1, 1))
    X_test["MMSE"] = mmse_scaler.transform(X_test["MMSE"].to_numpy().reshape(-1, 1))
    X_test["eTIV"] = mmse_scaler.transform(X_test["eTIV"].to_numpy().reshape(-1, 1))
    visit_3 = df[df["Visit_4.0"] == 1][["ID", "Y"]]
    y_train_final = []
    for id in X_train["ID"]:
        y_train_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_test_final = []
    for id in X_test["ID"]:
        y_test_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_train_super_final = pd.DataFrame({
    "Group": y_train_final})
    y_test_super_final = pd.DataFrame({
    "Group": y_test_final})
    X_test_final = X_test.drop(["ID"], axis = 1)
    X_train_final = X_train.drop(["ID"], axis = 1)
    rows_n = 5

    X_train_super_final = np.reshape(X_train_final.to_numpy(),(X_train_final.shape[0]//5,X_train_final.shape[1],rows_n))
    X_test_super_final = np.reshape(X_test_final.to_numpy(),(X_test_final.shape[0]//5,X_train_final.shape[1],rows_n))
    lstm_rnn = build_lstm_rnn((X_train_super_final.shape[1],X_train_super_final.shape[2]) )
    lstm_rnn.fit(X_train_super_final, y_train_super_final[0:int(len(X_train_super_final))], epochs = 100)
    not_final_lstm_rnn_prediction = lstm_rnn.predict(X_test_super_final)
    lstm_rnn_prediction = np.where(not_final_lstm_rnn_prediction > 0.5, 1, 0)
    lstm_rnn_conf = confusion_matrix(y_test_super_final[0:int(len(lstm_rnn_prediction))],lstm_rnn_prediction)
    lstm_rnn_plot_conf = ConfusionMatrixDisplay(lstm_rnn_conf)
    lstm_rnn_plot_conf.plot()
    TP=lstm_rnn_conf[1,1]
    FP=lstm_rnn_conf[0,1]
    TN=lstm_rnn_conf[0,0]
    FN=lstm_rnn_conf[1,0]
    accuracy=(TP+TN)/(TP+FP+FN+TN)
    recall =TP/(TP+FN)
    precision =TP/(TP+TN)
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)

print("Accuracy List:", accuracy_list)
print("Recall List:", recall_list)
print("Precision List:", precision_list)

In [ ]:
print("Accuracy List:", accuracy_list)


In [ ]:
print("Recall List:", recall_list)


In [ ]:
print("Precision List:", precision_list)

In [ ]:
mean_accuracy_list = statistics.mean(accuracy_list)
mean_recall_list = statistics.mean(recall_list)
mean_precision_list = statistics.mean(precision_list)
print("mean_accuracy:",mean_accuracy_list, "mean_recall:",mean_recall_list,"mean_precision:",mean_precision_list)

In [ ]:
inputshape=(X_train_super_final.shape[1],X_train_super_final.shape[2])


In [ ]:
def build_gru_rnn(input_shape):
    gru_rnn = tf.keras.Sequential()
    gru_rnn.add(tf.keras.layers.GRU(100, return_sequences = True,input_shape = True) )
    gru_rnn.add(tf.keras.layers.GRU(50,return_sequences = False))
    gru_rnn.add(tf.keras.layers.Dense(1,activation = "sigmoid"))
    gru_rnn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return gru_rnn

In [ ]:
for _ in range(5):
    N = 100
    m = 5

    Y = eta = np.zeros((N, m))
    Age = np.random.choice(range(age_range[0], age_range[1] + 1), size=N)
    MMSE = np.round(np.random.normal(mu_MMSE, sigma_MMSE, size=N))
    EDUC = np.round(np.random.normal(mu_EDUC, sigma_EDUC, size=N))
    Sex = np.random.binomial(1, 0.2, size=N)
    eTIV = np.round(np.random.normal(mu_eTIV, sigma_eTIV, size=N))
    ASF = np.random.normal(size=N)
    nWBV = np.random.normal(size=N)
    Visit = np.round(np.linspace(0, 4, num=m))

    B = np.array([0.1, -0.1, 0.2, -0.2, 0.3, 0.3, 0.1, 0.1, 0.2, 0.1])
    b = np.random.normal(0, 0.5, size=N)

    for i in range(N):
        for j in range(m):
            eta[i, j] = B[0] + B[1] * Visit[j] + B[2] * Age[i] + B[3] * MMSE[i] + B[4] * EDUC[i] + B[5] * Sex[i] + \
                        B[6] * eTIV[i] + B[7] * ASF[i] + B[8] * nWBV[i] + b[i]
            p = scipy.special.expit(eta[i, j])
            Y[i, j] = np.random.binomial(1, p)

   

    mat = np.empty((N * m, 10))
    p = 0
    for i in range(N):
        jk = 0
        for jj in range(p, p + m):
            mat[jj, 0] = i + 1
            mat[jj, 1] = Visit[jk]
            mat[jj, 2] = Age[i]
            mat[jj, 3] = MMSE[i]
            mat[jj, 4] = EDUC[i]
            mat[jj, 5] = Sex[i]
            mat[jj, 6] = eTIV[jk]
            mat[jj, 7] = ASF[jk]
            mat[jj, 8] = nWBV[jk]
            mat[jj, 9] = Y[i, jk]
            jk += 1
        p += m

    # Convert the `mat` array to a DataFrame
    df_mat = pd.DataFrame(mat, columns=['ID', 'Visit', 'Age', 'MMSE', 'EDUC', 'Sex', 'eTIV', 'ASF', 'nWBV', 'Y'])
    df = pd.get_dummies(df_mat, columns=["Visit"], prefix="Visit")
    RANDOM_STATE = 13
    splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state=RANDOM_STATE)
    split = splitter.split(df, groups=df['ID'])
    train_indexes, test_indexes = next(split)

    X_train = df.iloc[train_indexes]
    X_test = df.iloc[test_indexes]
    age_scaler = MinMaxScaler()
    educ_scaler = MinMaxScaler()
    mmse_scaler = MinMaxScaler()
    etiv_scaler = MinMaxScaler()

    age_scaler.fit(X_train["Age"].to_numpy().reshape(-1, 1))
    educ_scaler.fit(X_train["EDUC"].to_numpy().reshape(-1, 1))
    mmse_scaler.fit(X_train["MMSE"].to_numpy().reshape(-1, 1))
    etiv_scaler.fit(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
    X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
    X_train["MMSE"] = mmse_scaler.transform(X_train["MMSE"].to_numpy().reshape(-1, 1))
    X_train["eTIV"] = etiv_scaler.transform(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_test["Age"] = age_scaler.transform(X_test["Age"].to_numpy().reshape(-1, 1))
    X_test["EDUC"] = educ_scaler.transform(X_test["EDUC"].to_numpy().reshape(-1, 1))
    X_test["MMSE"] = mmse_scaler.transform(X_test["MMSE"].to_numpy().reshape(-1, 1))
    X_test["eTIV"] = mmse_scaler.transform(X_test["eTIV"].to_numpy().reshape(-1, 1))
    visit_3 = df[df["Visit_4.0"] == 1][["ID", "Y"]]
    y_train_final = []
    for id in X_train["ID"]:
        y_train_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_test_final = []
    for id in X_test["ID"]:
        y_test_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_train_super_final = pd.DataFrame({
    "Group": y_train_final})
    y_test_super_final = pd.DataFrame({
    "Group": y_test_final})
    X_test_final = X_test.drop(["ID"], axis = 1)
    X_train_final = X_train.drop(["ID"], axis = 1)
    rows_n = 5

In [ ]:

    X_train_super_final = np.reshape(X_train_final.to_numpy(),(X_train_final.shape[0]//5,X_train_final.shape[1],rows_n))
    X_test_super_final = np.reshape(X_test_final.to_numpy(),(X_test_final.shape[0]//5,X_train_final.shape[1],rows_n))

In [ ]:
X_train_super_final.shape[1]

In [ ]:
gru_rnn = build_gru_rnn((X_train_super_final.shape[1],X_train_super_final.shape[2]))
gru_rnn.fit(X_train_super_final, y_train_super_final[0:int(len(X_train_super_final))], epochs = 100)
not_final_gru_rnn_prediction = gru_rnn.predict(X_test_super_final)
gru_rnn_prediction = np.where(not_final_gru_rnn_prediction > 0.5, 1, 0)
print(classification_report(y_test_super_final,gru_rnn_prediction, target_names= ["Non-Demanted", "Demanted"]))
gru_rnn_conf = confusion_matrix(y_test_super_final[0:int(len(lstm_rnn_prediction))],gru_rnn_prediction)
gru_rnn_plot_conf = ConfusionMatrixDisplay(gru_rnn_conf)
gru_rnn_plot_conf.plot()

In [ ]:
for _ in range(5):
    N = 100
    m = 5

    Y = eta = np.zeros((N, m))
    Age = np.random.choice(range(age_range[0], age_range[1] + 1), size=N)
    MMSE = np.round(np.random.normal(mu_MMSE, sigma_MMSE, size=N))
    EDUC = np.round(np.random.normal(mu_EDUC, sigma_EDUC, size=N))
    Sex = np.random.binomial(1, 0.2, size=N)
    eTIV = np.round(np.random.normal(mu_eTIV, sigma_eTIV, size=N))
    ASF = np.random.normal(size=N)
    nWBV = np.random.normal(size=N)
    Visit = np.round(np.linspace(0, 4, num=m))

    B = np.array([0.1, -0.1, 0.2, -0.2, 0.3, 0.3, 0.1, 0.1, 0.2, 0.1])
    b = np.random.normal(0, 0.5, size=N)

    for i in range(N):
        for j in range(m):
            eta[i, j] = B[0] + B[1] * Visit[j] + B[2] * Age[i] + B[3] * MMSE[i] + B[4] * EDUC[i] + B[5] * Sex[i] + \
                        B[6] * eTIV[i] + B[7] * ASF[i] + B[8] * nWBV[i] + b[i]
            p = scipy.special.expit(eta[i, j])
            Y[i, j] = np.random.binomial(1, p)

   

    mat = np.empty((N * m, 10))
    p = 0
    for i in range(N):
        jk = 0
        for jj in range(p, p + m):
            mat[jj, 0] = i + 1
            mat[jj, 1] = Visit[jk]
            mat[jj, 2] = Age[i]
            mat[jj, 3] = MMSE[i]
            mat[jj, 4] = EDUC[i]
            mat[jj, 5] = Sex[i]
            mat[jj, 6] = eTIV[jk]
            mat[jj, 7] = ASF[jk]
            mat[jj, 8] = nWBV[jk]
            mat[jj, 9] = Y[i, jk]
            jk += 1
        p += m

    # Convert the `mat` array to a DataFrame
    df_mat = pd.DataFrame(mat, columns=['ID', 'Visit', 'Age', 'MMSE', 'EDUC', 'Sex', 'eTIV', 'ASF', 'nWBV', 'Y'])
    df = pd.get_dummies(df_mat, columns=["Visit"], prefix="Visit")
    RANDOM_STATE = 13
    splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state=RANDOM_STATE)
    split = splitter.split(df, groups=df['ID'])
    train_indexes, test_indexes = next(split)

    X_train = df.iloc[train_indexes]
    X_test = df.iloc[test_indexes]
    age_scaler = MinMaxScaler()
    educ_scaler = MinMaxScaler()
    mmse_scaler = MinMaxScaler()
    etiv_scaler = MinMaxScaler()

    age_scaler.fit(X_train["Age"].to_numpy().reshape(-1, 1))
    educ_scaler.fit(X_train["EDUC"].to_numpy().reshape(-1, 1))
    mmse_scaler.fit(X_train["MMSE"].to_numpy().reshape(-1, 1))
    etiv_scaler.fit(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_train["Age"] = age_scaler.transform(X_train["Age"].to_numpy().reshape(-1, 1))
    X_train["EDUC"] = educ_scaler.transform(X_train["EDUC"].to_numpy().reshape(-1, 1))
    X_train["MMSE"] = mmse_scaler.transform(X_train["MMSE"].to_numpy().reshape(-1, 1))
    X_train["eTIV"] = etiv_scaler.transform(X_train["eTIV"].to_numpy().reshape(-1, 1))

    X_test["Age"] = age_scaler.transform(X_test["Age"].to_numpy().reshape(-1, 1))
    X_test["EDUC"] = educ_scaler.transform(X_test["EDUC"].to_numpy().reshape(-1, 1))
    X_test["MMSE"] = mmse_scaler.transform(X_test["MMSE"].to_numpy().reshape(-1, 1))
    X_test["eTIV"] = mmse_scaler.transform(X_test["eTIV"].to_numpy().reshape(-1, 1))
    visit_3 = df[df["Visit_4.0"] == 1][["ID", "Y"]]
    y_train_final = []
    for id in X_train["ID"]:
        y_train_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_test_final = []
    for id in X_test["ID"]:
        y_test_final.append(visit_3[visit_3["ID"] == id]["Y"].values[0])

    y_train_super_final = pd.DataFrame({
    "Group": y_train_final})
    y_test_super_final = pd.DataFrame({
    "Group": y_test_final})
    X_test_final = X_test.drop(["ID"], axis = 1)
    X_train_final = X_train.drop(["ID"], axis = 1)
    rows_n = 5

    X_train_super_final = np.reshape(X_train_final.to_numpy(),(X_train_final.shape[0]//5,X_train_final.shape[1],rows_n))
    X_test_super_final = np.reshape(X_test_final.to_numpy(),(X_test_final.shape[0]//5,X_train_final.shape[1],rows_n))
    gru_rnn = build_gru_rnn((X_train_super_final.shape[1],X_train_super_final.shape[2]))
    gru_rnn.fit(X_train_super_final, y_train_super_final[0:int(len(X_train_super_final))], epochs = 100)
    not_final_gru_rnn_prediction = gru_rnn.predict(X_test_super_final)
    gru_rnn_prediction = np.where(not_final_gru_rnn_prediction > 0.5, 1, 0)
    print(classification_report(y_test_super_final,gru_rnn_prediction, target_names= ["Non-Demanted", "Demanted"]))
    gru_rnn_conf = confusion_matrix(y_test_super_final[0:int(len(lstm_rnn_prediction))],gru_rnn_prediction)
    gru_rnn_plot_conf = ConfusionMatrixDisplay(gru_rnn_conf)
    gru_rnn_plot_conf.plot()
    TP=lstm_rnn_conf[1,1]
    FP=lstm_rnn_conf[0,1]
    TN=lstm_rnn_conf[0,0]
    FN=lstm_rnn_conf[1,0]
    accuracy=(TP+TN)/(TP+FP+FN+TN)
    recall =TP/(TP+FN)
    precision =TP/(TP+TN)
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)

print("Accuracy List:", accuracy_list)
print("Recall List:", recall_list)
print("Precision List:", precision_list)